# Module 07: Window Functions and Advanced Transformations

**Difficulty**: ⭐⭐

**Estimated Time**: 85-100 minutes

**Prerequisites**: 
- Module 00: Introduction to Big Data and Spark Ecosystem
- Module 01: PySpark Setup and SparkSession
- Module 03: DataFrames and Datasets
- Module 05: DataFrame Operations
- Module 06: Spark SQL and Temporary Views

## Learning Objectives

By the end of this notebook, you will be able to:
1. Understand window functions and when to use them
2. Perform ranking operations (row_number, rank, dense_rank)
3. Calculate running totals and moving averages
4. Use lead() and lag() for accessing neighboring rows
5. Create and use User Defined Functions (UDFs) for custom transformations

## Setup

In [ ]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType
from pyspark.sql.functions import (
    col, lit, when,
    row_number, rank, dense_rank, percent_rank, ntile,
    lag, lead,
    sum, avg, count, min, max,
    round, concat, upper,
    year, month, dayofmonth, datediff, current_date
)
from pyspark.sql.types import StringType, IntegerType
from datetime import date, timedelta
import random

# Create SparkSession
spark = SparkSession.builder \
    .appName("Module 07: Window Functions and Advanced Transformations") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

print(f"✓ SparkSession created: {spark.sparkContext.appName}")
print(f"  Spark version: {spark.version}")
print(f"  Spark UI: {spark.sparkContext.uiWebUrl}")

### Create Sample DataFrames

In [ ]:
# Sample employees data
employees_data = [
    (1, "Alice Johnson", "Engineering", 75000, date(2020, 1, 15)),
    (2, "Bob Smith", "Sales", 65000, date(2019, 6, 10)),
    (3, "Charlie Brown", "Engineering", 80000, date(2021, 3, 22)),
    (4, "Diana Prince", "Marketing", 70000, date(2020, 8, 5)),
    (5, "Eve Davis", "Engineering", 85000, date(2018, 11, 30)),
    (6, "Frank Miller", "Sales", 68000, date(2022, 2, 14)),
    (7, "Grace Lee", "Marketing", 72000, date(2021, 9, 1)),
    (8, "Henry Wilson", "Engineering", 78000, date(2019, 4, 20)),
    (9, "Ivy Chen", "Sales", 71000, date(2020, 12, 10)),
    (10, "Jack Thompson", "Engineering", 82000, date(2021, 7, 5)),
    (11, "Kelly Adams", "Sales", 73000, date(2020, 3, 18)),
    (12, "Leo Martinez", "Marketing", 69000, date(2022, 1, 20))
]

df_employees = spark.createDataFrame(
    employees_data,
    ["emp_id", "name", "department", "salary", "hire_date"]
)

print("Employees DataFrame:")
df_employees.show()

# Sample sales data with dates
sales_data = []
base_date = date(2024, 1, 1)
products = ['Laptop', 'Phone', 'Tablet']
regions = ['North', 'South']

for i in range(30):
    sales_data.append((
        i + 1,
        base_date + timedelta(days=i),
        random.choice(products),
        random.choice(regions),
        random.randint(1, 5),
        round(random.uniform(500, 2000), 2)
    ))

df_sales = spark.createDataFrame(
    sales_data,
    ["sale_id", "sale_date", "product", "region", "quantity", "revenue"]
)

print("\nSales DataFrame (first 10 rows):")
df_sales.show(10)

# Sample stock prices
stock_data = []
for i in range(20):
    stock_data.append((
        base_date + timedelta(days=i),
        "TECH",
        round(100 + random.uniform(-10, 10), 2)
    ))

df_stocks = spark.createDataFrame(
    stock_data,
    ["date", "symbol", "price"]
)

print("\nStock Prices (first 10 rows):")
df_stocks.show(10)

## 1. Understanding Window Functions

### What are Window Functions?

**Window functions** perform calculations across a set of rows (a "window") related to the current row.

Unlike GROUP BY:
- **Rows are NOT collapsed** - you keep all original rows
- **Calculations across related rows** - e.g., running totals, rankings
- **Flexible partitioning and ordering**

### Window Function Components

```python
Window.partitionBy("column")     # Divide data into partitions
      .orderBy("column")         # Order within each partition
      .rowsBetween(start, end)   # Define window frame (optional)
```

### Visual Example

```
Original Data:
┌────────────┬────────┐
│ Department │ Salary │
├────────────┼────────┤
│ Sales      │ 65000  │
│ Sales      │ 68000  │
│ Sales      │ 71000  │
│ Engineering│ 75000  │
│ Engineering│ 80000  │
└────────────┴────────┘

With Window Function (row_number per department):
┌────────────┬────────┬──────┐
│ Department │ Salary │ Rank │
├────────────┼────────┼──────┤
│ Sales      │ 65000  │  1   │  ← Window: Sales rows
│ Sales      │ 68000  │  2   │  ←
│ Sales      │ 71000  │  3   │  ←
│ Engineering│ 75000  │  1   │  ← Window: Engineering rows
│ Engineering│ 80000  │  2   │  ←
└────────────┴────────┴──────┘
```

## 2. Ranking Functions

### row_number(), rank(), dense_rank()

| Function | Description | Example |
|----------|-------------|----------|
| `row_number()` | Sequential number, no gaps | 1, 2, 3, 4, 5 |
| `rank()` | Rank with gaps for ties | 1, 2, 2, 4, 5 |
| `dense_rank()` | Rank without gaps | 1, 2, 2, 3, 4 |

### Ranking Employees by Salary

In [ ]:
# Define window: partition by department, order by salary descending
window_spec = Window.partitionBy("department").orderBy(col("salary").desc())

# Apply ranking functions
df_ranked = df_employees.select(
    col("name"),
    col("department"),
    col("salary"),
    row_number().over(window_spec).alias("row_num"),
    rank().over(window_spec).alias("rank"),
    dense_rank().over(window_spec).alias("dense_rank")
)

print("Employee rankings by salary within department:")
df_ranked.orderBy("department", "row_num").show()

print("\n💡 Notice: row_number is always unique, rank has gaps, dense_rank has no gaps")

### Finding Top N per Group

In [ ]:
# Find top 2 highest paid employees per department
window_top = Window.partitionBy("department").orderBy(col("salary").desc())

df_top2 = df_employees \
    .withColumn("rank", row_number().over(window_top)) \
    .filter(col("rank") <= 2) \
    .select("name", "department", "salary", "rank")

print("Top 2 highest paid per department:")
df_top2.orderBy("department", "rank").show()

### Percent Rank and Ntile

In [ ]:
# percent_rank: Relative rank (0 to 1)
# ntile: Divide into N buckets

window_all = Window.orderBy(col("salary").desc())

df_percentiles = df_employees.select(
    col("name"),
    col("salary"),
    round(percent_rank().over(window_all), 2).alias("percent_rank"),
    ntile(4).over(window_all).alias("quartile")
)

print("Salary percentiles and quartiles:")
df_percentiles.orderBy("salary", ascending=False).show()

print("\n💡 Quartile 1 = top 25%, Quartile 4 = bottom 25%")

## 3. Aggregate Window Functions

### Running Totals

In [ ]:
# Calculate running total of revenue by date
window_running = Window.orderBy("sale_date").rowsBetween(Window.unboundedPreceding, Window.currentRow)

df_running_total = df_sales \
    .select(
        col("sale_date"),
        col("product"),
        col("revenue"),
        round(sum("revenue").over(window_running), 2).alias("running_total")
    ) \
    .orderBy("sale_date")

print("Running total of revenue:")
df_running_total.show(15)

print("\n💡 Running total accumulates revenue from start to current row")

### Moving Averages

In [ ]:
# Calculate 3-day moving average of stock prices
# Window: current row + 2 preceding rows
window_moving = Window.orderBy("date").rowsBetween(-2, 0)

df_moving_avg = df_stocks \
    .select(
        col("date"),
        col("symbol"),
        col("price"),
        round(avg("price").over(window_moving), 2).alias("moving_avg_3day")
    ) \
    .orderBy("date")

print("3-day moving average of stock price:")
df_moving_avg.show(10)

print("\n💡 Moving average smooths out short-term fluctuations")

### Cumulative Statistics

In [ ]:
# Multiple cumulative aggregations
window_cumulative = Window \
    .partitionBy("product") \
    .orderBy("sale_date") \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

df_cumulative = df_sales \
    .select(
        col("sale_date"),
        col("product"),
        col("revenue"),
        count("*").over(window_cumulative).alias("cumulative_count"),
        round(sum("revenue").over(window_cumulative), 2).alias("cumulative_revenue"),
        round(avg("revenue").over(window_cumulative), 2).alias("cumulative_avg")
    ) \
    .orderBy("product", "sale_date")

print("Cumulative statistics by product:")
df_cumulative.filter(col("product") == "Laptop").show(10)

## 4. Lag and Lead Functions

### What are lag() and lead()?

- **lag()**: Access previous row value
- **lead()**: Access next row value

Useful for:
- Comparing with previous/next period
- Calculating change/growth rates
- Finding consecutive patterns

In [ ]:
# Calculate day-over-day price change
window_ordered = Window.orderBy("date")

df_price_changes = df_stocks \
    .select(
        col("date"),
        col("price"),
        lag("price", 1).over(window_ordered).alias("prev_price"),
        lead("price", 1).over(window_ordered).alias("next_price")
    ) \
    .withColumn(
        "day_change",
        round(col("price") - col("prev_price"), 2)
    ) \
    .withColumn(
        "day_change_pct",
        round((col("price") - col("prev_price")) / col("prev_price") * 100, 2)
    )

print("Stock price changes:")
df_price_changes.show(10)

print("\n💡 First row has null prev_price (no previous data)")

### Finding Consecutive Patterns

In [ ]:
# Find consecutive days with price increase
window_price = Window.orderBy("date")

df_trends = df_stocks \
    .withColumn("prev_price", lag("price", 1).over(window_price)) \
    .withColumn(
        "is_increase",
        when(col("price") > col("prev_price"), 1).otherwise(0)
    ) \
    .select("date", "price", "prev_price", "is_increase")

print("Price increase indicators:")
df_trends.show(10)

## 5. Window Frames

### Understanding Window Frames

Window frames define **which rows** to include in the calculation.

**Common frames**:
```python
# From start to current row (cumulative)
.rowsBetween(Window.unboundedPreceding, Window.currentRow)

# Current row only
.rowsBetween(Window.currentRow, Window.currentRow)

# 3 rows: previous, current, next
.rowsBetween(-1, 1)

# Last 7 rows including current
.rowsBetween(-6, 0)

# All rows in partition
.rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
```

In [ ]:
# Compare different window frames
window_all = Window.orderBy("sale_date") \
    .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

window_cumulative = Window.orderBy("sale_date") \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

window_rolling3 = Window.orderBy("sale_date") \
    .rowsBetween(-2, 0)

df_frames = df_sales \
    .select(
        col("sale_date"),
        col("revenue"),
        round(avg("revenue").over(window_all), 2).alias("overall_avg"),
        round(avg("revenue").over(window_cumulative), 2).alias("cumulative_avg"),
        round(avg("revenue").over(window_rolling3), 2).alias("rolling_3day_avg")
    ) \
    .orderBy("sale_date")

print("Comparing different window frames:")
df_frames.show(10)

print("""
💡 Notice the differences:
   - overall_avg: Same for all rows (entire dataset)
   - cumulative_avg: Increases as more data accumulated
   - rolling_3day_avg: Changes with recent 3 days
""")

## 6. User Defined Functions (UDFs)

### What are UDFs?

**UDFs** let you define custom functions when built-in functions aren't enough.

⚠️ **Warning**: UDFs are slower than built-in functions!
- They break Catalyst optimization
- Require serialization/deserialization
- Use only when necessary

### Creating a Simple UDF

In [ ]:
# Define Python function
def categorize_salary(salary):
    if salary >= 80000:
        return "Senior"
    elif salary >= 70000:
        return "Mid-Level"
    else:
        return "Junior"

# Register as UDF
from pyspark.sql.functions import udf
categorize_salary_udf = udf(categorize_salary, StringType())

# Use UDF
df_with_category = df_employees.select(
    col("name"),
    col("salary"),
    categorize_salary_udf(col("salary")).alias("salary_category")
)

print("Using UDF to categorize salaries:")
df_with_category.show()

print("""
💡 Better alternative: Use built-in when() function!
   UDFs should be last resort.
""")

### Better Alternative: Built-in Functions

In [ ]:
# Same logic using built-in when() - MUCH FASTER!
df_with_category_builtin = df_employees.select(
    col("name"),
    col("salary"),
    when(col("salary") >= 80000, "Senior")
        .when(col("salary") >= 70000, "Mid-Level")
        .otherwise("Junior")
        .alias("salary_category")
)

print("Same result using built-in when():")
df_with_category_builtin.show()

print("✓ This is MUCH faster than UDF!")

### When to Use UDFs

✅ **Use UDFs when**:
- Complex business logic not expressible with built-in functions
- Need to call external libraries
- Integrating with existing Python code

❌ **Avoid UDFs when**:
- Built-in functions can do the job
- Performance is critical
- Working with large datasets

### UDF with Multiple Inputs

In [ ]:
# UDF with multiple inputs
def format_employee_info(name, department, salary):
    return f"{name} ({department}) - ${salary:,.0f}"

format_info_udf = udf(format_employee_info, StringType())

df_formatted = df_employees.select(
    col("name"),
    format_info_udf(
        col("name"),
        col("department"),
        col("salary")
    ).alias("formatted_info")
)

print("Formatted employee info:")
df_formatted.show(5, truncate=False)

## Exercises

### Exercise 1: Sales Rankings

Using the `df_sales` DataFrame:
1. Rank sales by revenue within each product category
2. Find the top 3 sales for each product
3. Add a column showing what percentage of total product revenue each sale represents
4. Display results sorted by product and rank

In [ ]:
# Exercise 1: Your code here

# Your code here

### Exercise 2: Moving Average Analysis

Using the `df_stocks` DataFrame:
1. Calculate 5-day moving average of price
2. Calculate 10-day moving average of price
3. Add a column indicating if price is above/below the 5-day moving average
4. Show dates where short-term trend (5-day) crosses above long-term trend (10-day)

In [ ]:
# Exercise 2: Your code here

# Your code here

### Exercise 3: Year-over-Year Comparison

Create sample monthly sales data for 2023 and 2024, then:
1. Calculate total revenue by month for each year
2. Use lag() to get previous year's revenue for same month
3. Calculate year-over-year growth percentage
4. Identify months with highest growth

In [ ]:
# Exercise 3: Your code here

# First create sample data for 2023 and 2024
# Then calculate YoY growth

# Your code here

### Exercise 4: Department Analytics

Using the `df_employees` DataFrame:
1. Calculate each employee's salary as percentage of department total
2. Find how each employee's salary compares to department average (difference)
3. Rank employees within department by hire date (earliest = 1)
4. Add a column showing the salary of the next person hired in the same department

In [ ]:
# Exercise 4: Your code here

# Your code here

## Solutions

### Exercise 1 Solution

In [ ]:
# Solution 1: Sales Rankings

# Window for ranking and percentage calculation
window_rank = Window.partitionBy("product").orderBy(col("revenue").desc())
window_total = Window.partitionBy("product") \
    .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

df_sales_ranked = df_sales \
    .withColumn("rank", row_number().over(window_rank)) \
    .withColumn("product_total_revenue", sum("revenue").over(window_total)) \
    .withColumn(
        "pct_of_product_revenue",
        round((col("revenue") / col("product_total_revenue")) * 100, 2)
    ) \
    .filter(col("rank") <= 3) \
    .select(
        "product",
        "sale_date",
        "revenue",
        "rank",
        "pct_of_product_revenue"
    ) \
    .orderBy("product", "rank")

print("Top 3 sales by product with revenue percentage:")
df_sales_ranked.show()

### Exercise 2 Solution

In [ ]:
# Solution 2: Moving Average Analysis

window_5day = Window.orderBy("date").rowsBetween(-4, 0)
window_10day = Window.orderBy("date").rowsBetween(-9, 0)

df_moving_avgs = df_stocks \
    .withColumn("ma_5day", round(avg("price").over(window_5day), 2)) \
    .withColumn("ma_10day", round(avg("price").over(window_10day), 2)) \
    .withColumn(
        "above_5day_ma",
        when(col("price") > col("ma_5day"), "Above").otherwise("Below")
    ) \
    .withColumn(
        "prev_ma_5day",
        lag("ma_5day", 1).over(Window.orderBy("date"))
    ) \
    .withColumn(
        "prev_ma_10day",
        lag("ma_10day", 1).over(Window.orderBy("date"))
    )

print("Stock price with moving averages:")
df_moving_avgs.select(
    "date", "price", "ma_5day", "ma_10day", "above_5day_ma"
).show(15)

# Find crossover points (5-day crosses above 10-day)
df_crossovers = df_moving_avgs.filter(
    (col("ma_5day") > col("ma_10day")) &
    (col("prev_ma_5day") <= col("prev_ma_10day"))
)

print("\nCrossover points (5-day MA crosses above 10-day MA):")
df_crossovers.select("date", "price", "ma_5day", "ma_10day").show()

### Exercise 3 Solution

In [ ]:
# Solution 3: Year-over-Year Comparison

# Create sample monthly data
monthly_data = []
for year in [2023, 2024]:
    for month in range(1, 7):  # Jan to Jun
        revenue = round(random.uniform(10000, 50000), 2)
        if year == 2024:
            revenue *= 1.15  # 15% growth on average
        monthly_data.append((year, month, revenue))

df_monthly = spark.createDataFrame(
    monthly_data,
    ["year", "month", "revenue"]
)

# Calculate YoY growth
window_yoy = Window.partitionBy("month").orderBy("year")

df_yoy = df_monthly \
    .withColumn("prev_year_revenue", lag("revenue", 1).over(window_yoy)) \
    .withColumn(
        "yoy_growth",
        round(
            ((col("revenue") - col("prev_year_revenue")) / col("prev_year_revenue")) * 100,
            2
        )
    ) \
    .orderBy("month", "year")

print("Year-over-Year revenue comparison:")
df_yoy.show()

print("\nMonths with highest growth:")
df_yoy.filter(col("yoy_growth").isNotNull()) \
    .orderBy(col("yoy_growth").desc()) \
    .show(3)

### Exercise 4 Solution

In [ ]:
# Solution 4: Department Analytics

# Windows for different calculations
window_dept = Window.partitionBy("department") \
    .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
window_hire = Window.partitionBy("department").orderBy("hire_date")

df_dept_analytics = df_employees \
    .withColumn("dept_total_salary", sum("salary").over(window_dept)) \
    .withColumn("dept_avg_salary", avg("salary").over(window_dept)) \
    .withColumn(
        "pct_of_dept_salary",
        round((col("salary") / col("dept_total_salary")) * 100, 2)
    ) \
    .withColumn(
        "diff_from_avg",
        round(col("salary") - col("dept_avg_salary"), 2)
    ) \
    .withColumn("hire_rank", row_number().over(window_hire)) \
    .withColumn("next_hire_salary", lead("salary", 1).over(window_hire)) \
    .select(
        "name",
        "department",
        "salary",
        "pct_of_dept_salary",
        "diff_from_avg",
        "hire_rank",
        "next_hire_salary"
    ) \
    .orderBy("department", "hire_rank")

print("Department analytics:")
df_dept_analytics.show(truncate=False)

## Summary

### Key Concepts Covered

✅ **Window Functions**: Calculations across related rows without collapsing

✅ **Ranking**: row_number(), rank(), dense_rank(), percent_rank(), ntile()

✅ **Aggregates**: sum(), avg(), count() over windows

✅ **Navigation**: lag(), lead() for accessing neighboring rows

✅ **UDFs**: Custom functions (use sparingly!)

### Window Function Components

```python
Window.partitionBy("col")              # Divide into groups
      .orderBy("col")                  # Sort within groups
      .rowsBetween(start, end)         # Define frame
```

### Common Window Frames

| Frame | Description | Use Case |
|-------|-------------|----------|
| `(unboundedPreceding, currentRow)` | From start to current | Running totals |
| `(-6, 0)` | Last 7 rows | 7-day moving average |
| `(-1, 1)` | Previous, current, next | Smoothing |
| `(unboundedPreceding, unboundedFollowing)` | All rows | Percentage of total |

### Ranking Functions

| Function | Behavior | Example |
|----------|----------|----------|
| `row_number()` | Sequential, no gaps | 1,2,3,4,5 |
| `rank()` | Gaps for ties | 1,2,2,4,5 |
| `dense_rank()` | No gaps | 1,2,2,3,4 |
| `percent_rank()` | 0 to 1 scale | 0, 0.25, 0.5, 0.75, 1.0 |
| `ntile(n)` | Split into n buckets | 1,1,2,2,3,3 |

### Lag and Lead

```python
lag("col", 1)   # Previous row value
lead("col", 1)  # Next row value
lag("col", 2)   # 2 rows back
lead("col", 3)  # 3 rows ahead
```

### UDF Best Practices

❌ **Avoid UDFs when possible**:
- They break Catalyst optimization
- Much slower than built-in functions
- Require serialization overhead

✅ **Use built-in functions instead**:
- `when()` for conditional logic
- String functions: `concat()`, `upper()`, `lower()`, `substring()`
- Math functions: `round()`, `abs()`, `sqrt()`
- Date functions: `year()`, `month()`, `datediff()`

✅ **Only use UDFs when**:
- Complex custom logic not in built-ins
- Calling external libraries
- Legacy Python code integration

### Common Use Cases

**Top N per Group**:
```python
Window.partitionBy("group").orderBy(col("value").desc())
df.withColumn("rank", row_number().over(window))
  .filter(col("rank") <= N)
```

**Running Total**:
```python
Window.orderBy("date")
      .rowsBetween(Window.unboundedPreceding, Window.currentRow)
df.withColumn("running_total", sum("amount").over(window))
```

**Moving Average**:
```python
Window.orderBy("date").rowsBetween(-6, 0)
df.withColumn("7day_avg", avg("value").over(window))
```

**Percentage of Total**:
```python
window_total = Window.partitionBy("group")
              .rowsBetween(unboundedPreceding, unboundedFollowing)
df.withColumn("total", sum("value").over(window_total))
  .withColumn("pct", col("value") / col("total") * 100)
```

**Day-over-Day Change**:
```python
Window.orderBy("date")
df.withColumn("prev_value", lag("value", 1).over(window))
  .withColumn("change", col("value") - col("prev_value"))
```

### Performance Tips

1. **Partition wisely**: More partitions = more parallelism
2. **Order only when needed**: Ordering is expensive
3. **Use specific frames**: Don't use entire partition if not needed
4. **Avoid UDFs**: Use built-in functions whenever possible
5. **Cache if reusing**: If same window used multiple times

### What's Next?

Congratulations! You've completed the PySpark fundamentals. Next steps:

- **Module 08**: Performance Optimization and Tuning
- **Module 09**: Advanced Topics (Broadcast joins, caching, partitioning)
- **Module 10**: Real-world Projects and Case Studies

### Additional Resources

- [Window Functions Guide](https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-window.html)
- [Window Functions API](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/window.html)
- [Built-in Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html)

In [ ]:
# Cleanup
spark.stop()
print("SparkSession stopped. ✓")